# Set-up and data preparation

In [1]:
from transformers import pipeline, AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import numpy as np
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
from sentence_transformers import SentenceTransformer, util, models
import os

C:\Users\dathn\anaconda3\envs\thesis_v2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch 
torch.cuda.is_available()

True

In [3]:
path_data = '..\\data_structured'

## Data loading

In [4]:
df_comb = pd.read_pickle(os.path.join(path_data, 'comb.pkl'))

In [75]:
sust_topics = [0,1,2,3,4,5,6,7,8,10,11,20,25]
lim_topics = [1,2,3,4,5,7,8,10,11]

In [6]:
df_analyze = df_comb[df_comb['topics'].isin(sust_topics)]

In [7]:
df_report = df_analyze[df_analyze['doc_type']=='report']
df_article = df_analyze[df_analyze['doc_type']=='news']
df_report.reset_index(drop = True, inplace = True)
df_article.reset_index(drop = True, inplace = True)

# Applying the pre-trained models

In [8]:
claim_checker = pipeline(model = "climatebert/environmental-claims",  device = 0, batch_size = 64)
sem_search = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
nli_model = "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli"

C:\Users\dathn\anaconda3\envs\thesis_v2\lib\site-packages\huggingface_hub-0.14.1-py3.8.egg\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dathn\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development


The claim verification model consists of three stages - claim identification, evidence sentence selection and finally inference analysis. The three models above will help us achieve these three tasks. The ClimateBERT model is pre-trained to detect environmental and climate claims, semantic search will help us identify the 5 most relevant sentences from the corpus and finally the actual model can be used to check the entailment.

First, we apply the ClimateBERT model to identify environmental claims:

In [ ]:
# first part of the pipeline - identifying claims
sentences = df_report['sentence'].tolist()  # Convert the column to a list

results = claim_checker(sentences)
df_report['claim'] = [result['label'] for result in results]
df_report['claim_probability'] = [result['score'] for result in results]

C:\Users\dathn\AppData\Local\Temp\ipykernel_8476\2836976421.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_report['claim'] = [result['label'] for result in results]
C:\Users\dathn\AppData\Local\Temp\ipykernel_8476\2836976421.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_report['claim_probability'] = [result['score'] for result in results]


In [11]:
df_report

,doc_type,company,sentence,word count,embeddings,char_length,anon_embeddings,topics,anon_sentence,claim,claim_probability
0,report,abb,One year into ABB's 2030 sustainability strate...,21,"[-0.03387668, 0.04702735, 0.0067782644, 0.0184...",123,"[-0.039394952, 0.055466365, 0.015991926, 0.016...",0,One year into the company 2030 sustainability ...,yes,0.947520
1,report,abb,"Compared with our baseline year of 2019, we ha...",27,"[0.072760716, 0.09948956, 0.077118196, 0.02812...",152,"[0.07276068, 0.09948958, 0.077118136, 0.028125...",1,"Compared with our baseline year of 2019, we ha...",yes,0.975864
2,report,abb,"Alongside these headline achievements, we made...",26,"[0.014213712, 0.033080414, 0.025592497, -0.021...",168,"[0.014213712, 0.033080414, 0.025592497, -0.021...",0,"Alongside these headline achievements, we made...",yes,0.982865
3,report,abb,Our 2030 GHG emissions reduction target was va...,28,"[0.0023520757, 0.040041316, 0.010221989, 0.016...",161,"[0.0023520836, 0.04004134, 0.010221971, 0.0164...",1,Our 2030 GHG emissions reduction target was va...,yes,0.990115
4,report,abb,We also joined the SBTi's Business Ambition fo...,36,"[-0.06374183, -0.02712268, -0.040378235, -0.02...",212,"[-0.06374184, -0.027122695, -0.040378183, -0.0...",1,We also joined the SBTi's Business Ambition fo...,yes,0.966742
...,...,...,...,...,...,...,...,...,...,...,...
30498,report,walmart,The initiative invites suppliers (starting wit...,31,"[-0.06545875, 0.027131714, -0.0116220275, 0.06...",228,"[-0.06545872, 0.027131697, -0.0116220135, 0.06...",10,The initiative invites suppliers (starting wit...,no,0.814833
30499,report,walmart,We also promote the adoption of best practices...,13,"[-0.02941792, -0.038456682, -0.01928836, -0.00...",81,"[-0.029417915, -0.038456634, -0.019288322, -0....",0,We also promote the adoption of best practices...,yes,0.792461
30500,report,walmart,"In FY2022, 87% of Walmart U.S. information, co...",25,"[-0.009387232, -0.017577449, -0.12768476, 0.02...",175,"[-0.03227741, -0.036879178, -0.1515622, 0.0158...",1,"In FY2022, 87% of the company U.S. information...",no,0.905760
30501,report,walmart,"To accelerate system-wide change, the Walmart ...",38,"[0.003541183, 0.046546437, -0.00028145174, -0....",260,"[-0.02246362, 0.022425106, -0.013262148, -0.02...",6,"To accelerate system-wide change, the the comp...",yes,0.913087


In [16]:
df_claims = df_report[df_report['claim']=='yes']
df_claims.reset_index(inplace = True, drop = True)
df_claims.shape

(12906, 11)

Now we create the sentence embeddings using the semantic search model. These embeddings will be used by the sentence transformers package to find the top 5 most similar sentences from the article corpus.

Since this took a while I will also pickle these to save my progress.

In [4]:
# df_article.to_pickle('art.pkl')
# df_claims.to_pickle('claims.pkl')

Sentence transformers has a utility called semantic search which can be used to do this:

In [13]:
%%time
for i,row in df_claims.iterrows():
    query_embedding = row['embeddings']
    company = row['company']
    # search only the article embeddings/sentences of the specific company
    corpus_embeddings = df_article[df_article['company']==company]['embeddings'].values
    top_5 = util.semantic_search(torch.Tensor(query_embedding), torch.Tensor(np.array(list(corpus_embeddings))), top_k = 5)
    break

CPU times: total: 0 ns
Wall time: 18.4 ms


In [14]:
top_5

[[{'corpus_id': 132, 'score': 0.7928017377853394},
  {'corpus_id': 524, 'score': 0.7137691974639893},
  {'corpus_id': 320, 'score': 0.7048271894454956},
  {'corpus_id': 609, 'score': 0.6708470582962036},
  {'corpus_id': 40, 'score': 0.6677325367927551}]]

Let us now create a new dataframe based on df_claims, which will store the same information as this dataframe, but will also additionally hold the top 5 most similar sentences in a separate column, as well as whether these sentences entail, contradict or are neutral towards each other. I use the MoritzLaurer NLI model for this purpose as it states that it is the best performing NLI model as of June 2022. The code used for the classification is mostly copied from the HuggingFace transformers website and modified for our purposes.

In [14]:
#df_entailment = df_entailment.reindex(df_entailment.columns.tolist() + ['top_sentences','predictions','probabilities'], axis=1)  # version > 0.20.0

In [172]:
# df_sample = df_entailment[:5] 


We repeat the same code as above but expand upon it further:

In [28]:
tokenizer = AutoTokenizer.from_pretrained(nli_model)
model = AutoModelForSequenceClassification.from_pretrained(nli_model).to(device)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [30]:
# # making lists to store values for the new columns
top_sentences_column = []
predictions = []
probabilities = []

# we run a for loop for each claim in the df_entailment dataset and check the validity of the claim
for i,row in df_claims.iterrows():
    # define our query (i.e. claim) and the company it's related to 
    query_embedding = row['embeddings']
    company = row['company']
    # search only the article embeddings/sentences of the specific company
    corpus_embeddings = df_article[df_article['company']==company]['embeddings'].values
    top_5 = util.semantic_search(torch.Tensor(query_embedding), torch.Tensor(np.array(list(corpus_embeddings))), top_k = 5)
    # define a list to hold our top sentences and predictions to add these as a new variable after the loop
    hard_predictions = []
    top_sentences = []
    soft_predictions =[]
    for sentence in  top_5[0]:
        # the premise is the claim
        premise = row['sentence']
        # the hypothesis is the sentence from the article(identified using the corpus id, which gives us the index of the sentence)
        hypothesis = df_article[df_article['company']==company]['sentence'].values[sentence['corpus_id']]
        tokens = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
        output = model(tokens["input_ids"].to(device))  # device = "cuda:0" or "cpu"
        soft_prediction = torch.softmax(output["logits"][0], -1)
        label_names = ["entailment", "neutral", "contradiction"]
        hard_prediction = label_names[torch.argmax(output["logits"][0], -1).item()]
        # append the different values to the correct list
        top_sentences.append(hypothesis)
        soft_predictions.append(max(torch.softmax(output["logits"][0], -1).tolist()))
        hard_predictions.append(hard_prediction)
    # # now add the different lists as new variables
    # df_sample.at[i,'top_sentences'] = str(top_sentences)
    # df_sample.at[i,'predictions'] = str(hard_predictions)
    # df_sample.at[i,'probabilities'] = str(soft_predictions)
    top_sentences_column.append(top_sentences)
    predictions.append(hard_predictions)
    probabilities.append(soft_predictions)

df_claims['top_sentences'] = top_sentences_column
df_claims['predictions'] = predictions
df_claims['probabilities'] = probabilities

C:\Users\dathn\AppData\Local\Temp\ipykernel_8476\1041120475.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_claims['top_sentences'] = top_sentences_column
C:\Users\dathn\AppData\Local\Temp\ipykernel_8476\1041120475.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_claims['predictions'] = predictions
C:\Users\dathn\AppData\Local\Temp\ipykernel_8476\1041120475.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [33]:
df_claims.to_pickle('entailment.pkl')

In [38]:
from collections import Counter

def most_frequent_category(categories_list):
    counter = Counter(categories_list)
    return counter.most_common(1)[0][0]

In [66]:
cat_list = ['neutral','neutral','entailment','neutral','contradiction']
count_non_neutral(cat_list)

'neutral'

In [63]:
def count_non_neutral(categories_list):
    counter = Counter(categories_list)
    for element,count in counter.items():
        if (count >= 2) & (element!='neutral'):
            return element
    return 'neutral'
        

In [39]:
df_claims['consensus'] = df_claims['predictions'].apply(most_frequent_category)
df_claims['consensus'].value_counts()

C:\Users\dathn\AppData\Local\Temp\ipykernel_8476\4136441315.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_claims['consensus'] = df_claims['predictions'].apply(most_frequent_category)


neutral          12796
contradiction       71
entailment          39
Name: consensus, dtype: int64

In [77]:
df_claims['consensus'] = df_claims['predictions'].apply(count_non_neutral)

C:\Users\dathn\AppData\Local\Temp\ipykernel_8476\1825575629.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_claims['consensus'] = df_claims['predictions'].apply(count_non_neutral)


In [78]:
df_claims

,doc_type,company,sentence,word count,embeddings,char_length,anon_embeddings,topics,anon_sentence,claim,claim_probability,top_sentences,predictions,probabilities,consensus,most_similar_sentence
0,report,abb,One year into ABB's 2030 sustainability strate...,21,"[-0.03387668, 0.04702735, 0.0067782644, 0.0184...",123,"[-0.039394952, 0.055466365, 0.015991926, 0.016...",0,One year into the company 2030 sustainability ...,yes,0.947520,"[Last year, ABB released its Sustainability St...","[neutral, entailment, neutral, neutral, neutral]","[0.9908430576324463, 0.965498685836792, 0.7367...",neutral,neutral
1,report,abb,"Compared with our baseline year of 2019, we ha...",27,"[0.072760716, 0.09948956, 0.077118196, 0.02812...",152,"[0.07276068, 0.09948958, 0.077118136, 0.028125...",1,"Compared with our baseline year of 2019, we ha...",yes,0.975864,[SN: Company report shows that ABB's greenhous...,"[neutral, neutral, neutral, neutral, contradic...","[0.999405026435852, 0.994537889957428, 0.99930...",neutral,neutral
2,report,abb,"Alongside these headline achievements, we made...",26,"[0.014213712, 0.033080414, 0.025592497, -0.021...",168,"[0.014213712, 0.033080414, 0.025592497, -0.021...",0,"Alongside these headline achievements, we made...",yes,0.982865,[We've been making concrete efforts towards ac...,"[neutral, neutral, neutral, neutral, neutral]","[0.9995107650756836, 0.9830822944641113, 0.995...",neutral,neutral
3,report,abb,Our 2030 GHG emissions reduction target was va...,28,"[0.0023520757, 0.040041316, 0.010221989, 0.016...",161,"[0.0023520836, 0.04004134, 0.010221971, 0.0164...",1,Our 2030 GHG emissions reduction target was va...,yes,0.990115,[Our greenhouse gas emissions reduction target...,"[entailment, neutral, neutral, neutral, neutral]","[0.9383401274681091, 0.8523229956626892, 0.999...",neutral,neutral
4,report,abb,We also joined the SBTi's Business Ambition fo...,36,"[-0.06374183, -0.02712268, -0.040378235, -0.02...",212,"[-0.06374184, -0.027122695, -0.040378183, -0.0...",1,We also joined the SBTi's Business Ambition fo...,yes,0.966742,[Although borne from the electrification busin...,"[neutral, neutral, neutral, neutral, neutral]","[0.9956797361373901, 0.9982929825782776, 0.999...",neutral,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12901,report,walmart,Our Responsible Sourcing program sets expectat...,34,"[-0.06368477, 0.010128789, -0.021725688, -0.02...",257,"[-0.06368478, 0.010128791, -0.02172569, -0.022...",0,Our Responsible Sourcing program sets expectat...,yes,0.645536,[This collaboration supports the industry thro...,"[neutral, neutral, neutral, neutral, neutral]","[0.995686948299408, 0.9650912880897522, 0.9904...",neutral,neutral
12902,report,walmart,Walmart has prioritized working with stakehold...,34,"[0.0054569603, 0.021002548, 0.0049035875, 0.00...",258,"[-0.027358495, 0.019750984, -0.0058520334, 0.0...",0,the company has prioritized working with stake...,yes,0.599842,"[To promote human dignity, Walmart has also co...","[neutral, neutral, neutral, neutral, neutral]","[0.9997122883796692, 0.994895875453949, 0.9991...",neutral,neutral
12903,report,walmart,To advance responsible recruitment across our ...,23,"[-0.044291113, -0.04852808, -0.02642449, 0.013...",166,"[-0.06653639, -0.061157897, -0.023344118, 0.01...",0,To advance responsible recruitment across our ...,yes,0.825599,[Walmart recently held its seventh annual Supp...,"[contradiction, neutral, neutral, neutral, neu...","[0.7207563519477844, 0.9993495345115662, 0.995...",neutral,neutral
12904,report,walmart,We also promote the adoption of best practices...,13,"[-0.02941792, -0.038456682, -0.01928836, -0.00...",81,"[-0.029417915, -0.038456634, -0.019288322, -0....",0,We also promote the adoption of best practices...,yes,0.792461,"[We work closely with logistics partners, NGOs...","[neutral, neutral, neutral, neutral, neutral]","[0.9987469911575317, 0.9983007311820984, 0.997...",neutral,neutral


In [80]:
df_claims[df_claims['topics'].isin(lim_topics)]['consensus'].value_counts()

neutral          10435
contradiction      242
entailment         190
Name: consensus, dtype: int64

In [49]:
df_claims[df_claims['consensus'] == 'contradiction']

,doc_type,company,sentence,word count,embeddings,char_length,anon_embeddings,topics,anon_sentence,claim,claim_probability,top_sentences,predictions,probabilities,consensus
335,report,airbus,CO2: reduce direct (scope 1) and indirect (sco...,19,"[0.042349424, 0.10064642, 0.03514003, 0.009380...",103,"[0.042349424, 0.10064642, 0.03514003, 0.009380...",1,CO2: reduce direct (scope 1) and indirect (sco...,yes,0.901820,[We're focused on carbon emission reduction of...,"[contradiction, contradiction, contradiction, ...","[0.9953846335411072, 0.5285486578941345, 0.643...",contradiction
396,report,airbus,SAF produced by using most advanced pathways c...,20,"[-0.006607856, -0.035302263, -0.07366908, 0.02...",122,"[-0.0066078347, -0.035302337, -0.0736691, 0.02...",8,SAF produced by using most advanced pathways c...,yes,0.990116,[SAF produced using the most advanced pathways...,"[contradiction, entailment, contradiction, neu...","[0.9935458302497864, 0.9918025135993958, 0.864...",contradiction
468,report,apple,213 suppliers committed to renewable electrici...,28,"[-0.0044368203, -0.008024681, 0.02283219, -0.0...",202,"[-0.019847926, -0.008363879, -0.02822676, -0.0...",3,213 suppliers committed to renewable electrici...,yes,0.994511,"[As of the report date, 213 of the company's m...","[entailment, contradiction, neutral, contradic...","[0.8047620058059692, 0.9574432969093323, 0.998...",contradiction
469,report,apple,"In fiscal year 2021, Apple and its suppliers b...",27,"[-0.035300363, 0.025236329, 0.055155836, 0.037...",159,"[-0.08419936, 0.03195669, 0.010630614, 0.05460...",3,"In fiscal year 2021, the company and its suppl...",yes,0.994993,[The projects will generate 1.2 gigawatts of r...,"[contradiction, contradiction, entailment, neu...","[0.8817325830459595, 0.9783431887626648, 0.935...",contradiction
584,report,apple,"Together, these new initiatives reduced total ...",26,"[0.023158113, 0.11676854, 0.08528477, 0.051681...",156,"[0.023158146, 0.11676853, 0.085284784, 0.05168...",1,"Together, these new initiatives reduced total ...",yes,0.994826,[This change will effectively avoid 18 million...,"[contradiction, contradiction, contradiction, ...","[0.9969239830970764, 0.5747354626655579, 0.996...",contradiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11648,report,tesco,"In 2009, Tesco became the first business globa...",18,"[0.015835347, 0.041475583, -0.042232767, 0.046...",100,"[-0.012548159, 0.027339203, -0.026353557, 0.03...",1,"In 2009, the company became the first business...",yes,0.992157,"[Last year, Tesco committed to net-zero emissi...","[contradiction, contradiction, contradiction, ...","[0.867167592048645, 0.9900526404380798, 0.7065...",contradiction
11669,report,tesco,"targets in line with a 1.5 C trajectory, commi...",19,"[0.0045745503, -0.029099684, -0.0049787, 0.024...",102,"[0.004574552, -0.029099733, -0.004978734, 0.02...",1,"targets in line with a 1.5 C trajectory, commi...",yes,0.992899,[Building on the good progress we've made in c...,"[contradiction, contradiction, neutral, contra...","[0.9856816530227661, 0.9069458842277527, 0.998...",contradiction
11978,report,totalenergies,"Republic of the Congo In March 2021, TotalEner...",39,"[-0.07422443, 0.015132636, -0.009076298, -0.00...",249,"[-0.063250385, 0.01810402, -0.024252135, 0.009...",1,"Republic of the Congo In March 2021, the compa...",yes,0.994960,[TOTALENERGIES has signed Corporate Power Purc...,"[neutral, contradiction, contradiction, neutra...","[0.5680773854255676, 0.8769581317901611, 0.969...",contradiction
12165,report,toyota,"Toyota revealed in the CDP2 A List, the highes...",26,"[0.016821591, -0.01882537, 0.0045505436, -0.02...",148,"[-0.012330274, -0.040687118, -0.025833614, -0....",6,"the company revealed in the CDP2 A List, the h...",yes,0.650870,[According to an article published by Electrek...,"[contradiction, contradiction, neutral, contra...","[0.9970995187759399, 0.9604046940803528, 0.999...",contradiction


In [50]:
df_claims.iloc[12165]['top_sentences']

["According to an article published by Electrek, a recent Greenpeace study ranked Toyota last among the world's top 10 automotive companies as far as its decarbonization efforts are concerned.",
 "Sure, Toyota is behind nine other companies, but it arguably shouldn't be on the list at all.",
 "This is the organization's highest rating, and one that acknowledges the standard Toyota Safety Sense 2.5+ system that includes:",
 "The world's top-selling carmaker Toyota has come joint last in a Greenpeace ranking of carbon emission efforts by auto firms, according to a list published Thursday during the COP26 climate summit.",
 'It may come as a surprise for some fans of Toyota and its fuel-efficient hybrid vehicles to learn that it ranks near Chevron and Exxon Mobil when it comes to its overall climate policy footprint.']

In [51]:
df_claims.iloc[12165]['sentence']

'Toyota revealed in the CDP2 A List, the highest rank, in both the climate change and water security categories scored by CDP (2016 2017, 2019 2020).'

In [82]:
gw_database_total = {}
for firm in set(df_claims['company']):
     # calculate the average sentiment score for this firm for symbolic actions
    ver_score = len(df_claims[(df_claims['company']==firm)&(df_claims['consensus']!='entailment')])/len(df_claims[df_claims['company']==firm])
    gw_database_total[firm] = ver_score

In [83]:
gw_database_total

{'google': 0.9876543209876543,
 'hershey': 0.9802631578947368,
 'veolia': 1.0,
 'colgate': 0.9957627118644068,
 'dupont': 1.0,
 'shell': 0.9759036144578314,
 'microsoft': 0.9877862595419847,
 'exxon': 1.0,
 'inditex': 1.0,
 'coca-cola': 0.9698275862068966,
 'dell': 0.9761904761904762,
 'cemex': 0.9826839826839827,
 'airbus': 0.9696969696969697,
 'tesla': 1.0,
 'boeing': 0.9930555555555556,
 'general-mills': 0.9716981132075472,
 'bayer': 0.9824561403508771,
 'nike': 0.9961832061068703,
 'renault': 0.976,
 'walmart': 1.0,
 'mitsubishi': 0.993006993006993,
 'apple': 0.9437689969604863,
 'volvo': 0.953125,
 'edp': 1.0,
 'honda': 0.9885057471264368,
 'tesco': 1.0,
 'adidas': 0.981651376146789,
 'citi': 1.0,
 'abb': 0.9897959183673469,
 'blackrock': 1.0,
 'hp': 0.9855072463768116,
 'ibm': 0.9636363636363636,
 'hyundai': 0.9868852459016394,
 'linde': 0.9897959183673469,
 'beiersdorf': 1.0,
 'h&m': 0.963076923076923,
 'mercedes': 0.988795518207283,
 'p&g': 0.9924242424242424,
 'pepsico': 0.954

In [90]:
from sklearn.preprocessing import MinMaxScaler

df_gw_total = pd.DataFrame(gw_database_total.items(),columns=['company', 'verification_score'])
scaler = MinMaxScaler()
df_gw_total['verification_score'] = scaler.fit_transform(df_gw_total[['verification_score']])

In [93]:
df_gw_total.to_csv('verification_scores.csv', index = False)

In [92]:
df_claims_lim = df_claims[df_claims['topics'].isin(lim_topics)]
df_claims_lim.reset_index(inplace = True, drop = True)
df_claims_lim.shape

(10867, 16)

In [94]:
gw_database_total = {}
for firm in set(df_claims_lim['company']):
     # calculate the average sentiment score for this firm for symbolic actions
    ver_score = len(df_claims_lim[(df_claims_lim['company']==firm)&(df_claims_lim['consensus']!='entailment')])/len(df_claims_lim[df_claims_lim['company']==firm])
    gw_database_total[firm] = ver_score

In [95]:
df_gw_lim = pd.DataFrame(gw_database_total.items(),columns=['company', 'verification_score_lim'])
df_gw_lim['verification_score_lim'] = scaler.fit_transform(df_gw_lim[['verification_score_lim']])

In [104]:
df_verification = pd.merge(df_gw_total, df_gw_lim)
df_verification.to_csv('verification_scores.csv', index = False)

In [103]:
df_gw_lim

,company,verification_score_lim
0,google,0.753731
1,hershey,0.638686
2,veolia,1.000000
3,colgate,0.915385
4,dupont,1.000000
...,...,...
58,starbucks,0.616279
59,totalenergies,0.838762
60,volkswagen,0.744845
61,komatsu,1.000000


In [87]:
# tokenizer = AutoTokenizer.from_pretrained(nli_model)
# model = AutoModelForSequenceClassification.from_pretrained(nli_model)

# for sentence in top_5[0]:
#     premise = row['sentence']
#     hypothesis = df_article[df_article['company']==company]['sentence'].values[sentence['corpus_id']]
#     tokens = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
#     output = model(tokens["input_ids"].to(device))  # device = "cuda:0" or "cpu"
#     prediction = torch.softmax(output["logits"][0], -1).tolist()
#     label_names = ["entailment", "neutral", "contradiction"]
#     prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
#     print(prediction)

{'entailment': 0.3, 'neutral': 99.7, 'contradiction': 0.1}
{'entailment': 0.1, 'neutral': 99.6, 'contradiction': 0.3}
{'entailment': 0.1, 'neutral': 99.6, 'contradiction': 0.3}
{'entailment': 0.2, 'neutral': 1.7, 'contradiction': 98.1}
{'entailment': 0.2, 'neutral': 99.2, 'contradiction': 0.6}


In [ ]:
# # in case the input sentence is too long:
# input_id_chunks = tokens_plus['input_ids'][0].split(510)
# mask_chunks = tokens_plus['attention_mask'][0].split(510)

# input_id_chunks = list(input_id_chunks)
# mask_chunks = list(mask_chunks)


# chunksize = 512
# for i in range(len(input_id_chunks)):
#     input_id_chunks[i] = torch.cat([
#         torch.Tensor([101]), input_id_chunks[i], torch.Tensor([102])
#     ])
#     mask_chunks[i] = torch.cat([
#         torch.Tensor([1]), mask_chunks[i], torch.Tensor([1])
#     ])
#     pad_len = chunksize - input_id_chunks[i].shape[0]
    
#     if pad_len > 0:
#         input_id_chunks[i] = torch.cat([
#             input_id_chunks[i], torch.Tensor([0]*pad_len)
#         ])
#         mask_id_chunks[i] = torch.cat([
#             mask_id_chunks[i], torch.Tensor([0]*pad_len)
#         ])

In [ ]:
# input_ids = torch.stack(input_id_chunks)
# attention_mask = torch.stack(mask_chunks)

# input_dict = {
#     'input_ids':input_ids.long(),
#     'attention_mask': attention_mask.int()
# }
# input_dict

In [ ]:
# outputs = model(**input_dict)

# probs = torch.nn.functional.softmax(outputs[0], dim = -1)
# probs

In [ ]:
# mean = probs.mean(dim = 0)